In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

Région possible :
- all
- north-america
- europe
- brazil
- asia-pacific
- korea
- japan
- latin-america
- oceania
- mena
- game-changers
- collegiate

In [2]:
url = "https://www.vlr.gg/events/europe"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<title>
			Valorant esports Events and Tournaments | VLR.gg		</title>
<meta content="Schedule and list of ongoing competitive Valorant tournaments, leagues, and events" name="description"/>
<meta charset="utf-8"/>
<meta content="Valorant esports Events and Tournaments | VLR.gg" property="og:title"/>
<meta content="VLR.gg" property="og:site_name"/>
<meta content="Schedule and list of ongoing competitive Valorant tournaments, leagues, and events" property="og:description"/>
<meta content="/img/vlr/favicon.png" property="og:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="@vlrdotgg" name="twitter:site"/>
<meta content="Valorant esports Events and Tournaments | VLR.gg" name="twitter:title"/>
<meta content="Schedule and list of ongoing competitive Valorant tournaments, leagues, and events" name="twitter:description"/>
<meta content="https://www.vlr.gg/img/vlr/logo_tw.png" name="twitter:image"/>
<meta content="#000" name="theme-co

In [3]:
def upcoming(balise):
    return balise.name == 'a' and balise.find('span', class_='event-item-desc-item-status mod-upcoming') is not None

upcomings = pd.DataFrame(columns=['Match','Name','Date'])

# Utiliser la fonction personnalisée pour récupérer les balises <a>
balises_a = soup.find_all(upcoming)

# Afficher les balises <a> trouvées
for balise_a in balises_a:
    texts = balise_a.find("div",class_="event-item-title")
    text = texts.get_text(strip=True)
    date = balise_a.find("div", class_="event-item-desc-item mod-dates").get_text(strip=True).replace('Dates', '')
    if "—" in date:
        periode1, periode2 = date.split("—")
        if len(periode2) <= 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            jour2 = periode2
            date = f"{jour1} {mois1} - {jour2} {mois1}"
        elif periode2 == "TBD":
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2 = periode2
            date = f"{jour1} {mois1} - {mois2}"
        elif len(periode2) > 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2, jour2 = [x.strip() for x in periode2.split()]
            date = f"{jour1} {mois1} - {jour2} {mois2}"
    else:
        mois1, jour1 = [x.strip() for x in periode1.split()]
        date = f"{jour1} {mois1}"
    url = balise_a.get('href')
    match = re.search(r"/event/(\d+)", url)
    match = match.group(1)
    upcomings = upcomings._append({'Match':match, 'Name':text, 'Date':date}, ignore_index=True)
    
upcomings

,Match,Name,Date
0,1874,VALORANT East: United: Season 2: Stage 3 - Finals,2 Nov - 10 Nov
1,1813,VALORANT Challengers 2023 Portugal: Drift,4 Nov - 26 Nov
2,1861,Spike Nations #4,8 Nov - 16 Dec


In [4]:
def ongoing(balise):
    return balise.name == 'a' and balise.find('span', class_='event-item-desc-item-status mod-ongoing') is not None

ongoings = pd.DataFrame(columns=['Match', 'Name', 'Date'])

# Utiliser la fonction personnalisée pour récupérer les balises <a>
balises_a = soup.find_all(ongoing)

# Afficher les balises <a> trouvées
for balise_a in balises_a:
    texts = balise_a.find("div",class_="event-item-title")
    text = texts.get_text(strip=True)
    date = balise_a.find("div", class_="event-item-desc-item mod-dates").get_text(strip=True).replace('Dates', '')
    if "—" in date:
        periode1, periode2 = date.split("—")
        if len(periode2) <= 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            jour2 = periode2
            date = f"{jour1} {mois1} - {jour2} {mois1}"
        elif periode2 == "TBD":
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2 = periode2
            date = f"{jour1} {mois1} - {mois2}"
        elif len(periode2) > 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2, jour2 = [x.strip() for x in periode2.split()]
            date = f"{jour1} {mois1} - {jour2} {mois2}"
    else:
        mois1, jour1 = [x.strip() for x in periode1.split()]
        date = f"{jour1} {mois1}"
    url = balise_a.get('href')
    match = re.search(r"/event/(\d+)", url)
    match = match.group(1)
    ongoings = ongoings._append({'Match':match,'Name':text, 'Date':date}, ignore_index=True)
    
ongoings

,Match,Name,Date
0,1812,Crossfire Cup 2023,17 Sep - 11 Nov
1,1855,Norwegian Valorant League Høsten 2023,19 Oct - 10 Dec
2,1786,Project V 2023: THE POKAL,30 Sep - 4 Nov
3,1836,VALORANT Challengers 2023 Italy: ON // THE BAT...,14 Oct - 3 Nov
4,1770,GAMEON VALORANT Tournament,4 Sep - 16 Sep
5,1778,VALORANT Challengers 2023 DACH: Arcade,16 Sep - 26 Nov


In [5]:
def completed(balise):
    return balise.name == 'a' and balise.find('span', class_='event-item-desc-item-status mod-completed') is not None

completeds = pd.DataFrame(columns=['Match','Name', 'Date'])

# Utiliser la fonction personnalisée pour récupérer les balises <a>
balises_a = soup.find_all(completed)

# Afficher les balises <a> trouvées
for balise_a in balises_a:
    texts = balise_a.find("div",class_="event-item-title")
    text = texts.get_text(strip=True)
    date = balise_a.find("div", class_="event-item-desc-item mod-dates").get_text(strip=True).replace('Dates', '')
    if "—" in date:
        periode1, periode2 = date.split("—")
        if len(periode2) <= 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            jour2 = periode2
            date = f"{jour1} {mois1} - {jour2} {mois1}"
        elif periode2 == "TBD":
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2 = periode2
            date = f"{jour1} {mois1} - {mois2}"
        elif len(periode2) > 2:
            mois1, jour1 = [x.strip() for x in periode1.split()]
            mois2, jour2 = [x.strip() for x in periode2.split()]
            date = f"{jour1} {mois1} - {jour2} {mois2}"
    else:
        mois1, jour1 = [x.strip() for x in periode1.split()]
        date = f"{jour1} {mois1}"
    url = balise_a.get('href')
    match = re.search(r"/event/(\d+)", url)
    match = match.group(1)
    completeds = completeds._append({'Match':match,'Name':text, 'Date':date}, ignore_index=True)
 
completeds

,Match,Name,Date
0,1866,VALORANT East: United: Season 2: Stage 3 - Las...,26 Oct - 29 Oct
1,1877,Türkiye 100th Anniversary VALORANT Cup Finals,27 Oct - 29 Oct
2,1879,EPIC.LAN #40,27 Oct - 29 Oct
3,1872,BBL-FUT 100. Yıl Gösteri Maçı Showmatch,27 Oct
4,1873,Golden Goose #16,28 Oct - 29 Oct
...,...,...,...
186,1298,EPIC.LAN #37 - BEACON Split 2,28 Oct - 31 Oct
187,1292,Valorant East: United - Stage 3 - Monthly Cup #2,29 Oct - 30 Oct
188,1295,Bad Omen,29 Oct
189,1726,Project V Division 1 - Split 2: Stage 3,6 Aug - TBD


In [6]:
completeds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191 entries, 0 to 190
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Match   191 non-null    object
 1   Name    191 non-null    object
 2   Date    191 non-null    object
dtypes: object(3)
memory usage: 4.6+ KB
